In [1]:
from jax import config
config.update("jax_enable_x64", True)

import jax.numpy as np 
import numpy as onp
from jax import jacfwd, grad, jit, vmap, hessian
from jaxlie import SE2, SO2, manifold
import meshcat
from meshcat import transformations as tfm
from meshcat import geometry as geom 
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output

def euler_step(x, f, dt, *args):
    return x + f(x, *args) * dt

def rk4_step(x, f, dt, *args):
    # one step of runge-kutta integration
    k1 = dt * f(x, *args)
    k2 = dt * f(x + k1/2, *args)
    k3 = dt * f(x + k2/2, *args)
    k4 = dt * f(x + k3, *args)
    return x + 1/6 * (k1 + 2 * k2 + 2 * k3 + k4)

# 2D Planar Drone Backflip using Direct methods and Direct Shooting MPC

In this homework, we will be controlling a 2D drone to perform a backflip maneuver using direct optimal control in the style of MPC!


## Dynamics Model 
We will be using a 2D (planar) drone system with two force inputs at each robot (see below).

![drone](quadrotor2d.png)


The manipulator equation of the drone are given as 
\begin{align*}
    m \ddot{p_x} &= - (u_1 + u_2) \sin(\theta) \\ 
    m \ddot{p_y} &= (u_1 + u_2) \cos(\theta) - m g \\ 
    I \ddot{\theta} &= r (u_1 - u_2)
\end{align*}
where $m$ is the mass, $I$ is the inertia, $g$ is gravity, $r$ is the distance from the center to the base of the propeller, and the state is given as $x=[p_x, p_y, \theta, \dot{p_x}, \dot{p_y}, \dot{\theta}]$.
The degrees of freedom at the $x,y$ position, drone rotation relative to the world $\theta$, and there are two force inputs $u_1, u_2$ for each rotor. 

As we are using a direct transcription approach, we need to write the dynamics as a discrete-time system, i.e., $x[k+1] = x[k] + dt * f(x[k], u[k])$

In [2]:
_dt = 0.1
_g  = 9.81 
_c1 = 0.02 
_c2 = 0.02
_r = 0.1
_I = 0.1
_m = 0.1

def f(x, u):
    """
        Input: state x=[px,py, theta, pxdt, pydt, thetadt], control u = [u1, u2]
        output: \dot{x} 
    """
    px, py, th, pxt, pyt, tht = x
    u1, u2 = u
    F = u1+u2 
    T = u1-u2
    xtt = - F * np.sin(th) / _m 
    ytt = F * np.cos(th) / _m - _g
    thtt =  _r * T/_I
    return np.array([pxt, pyt, tht, xtt, ytt, thtt])

@jit
def F(x, u):
    """
        solves x[t+dt] = x[t] + dt * f(x[t], u[t])
    """
    return euler_step(x, f, _dt, u)

# MPC-based solution of drone backflip

Here, we will solve the same problem but instead running an MPC controller instead of trajectory optimization. We will make several reductions to the problem above to make the MPC run faster. 

## Q6 10pts: Soft control constraints

First we will reduce the problem to a shooting based objective with soft constraints. Let's construct a function that, given an initial state condition and a control sequence returns a state and control trajectory. We will also incorporate a soft control constraint using a differentiable saturation function `umax*(tanh(x)/2 +0.5)` into the forward shooting which returns a smooth approximation of a function clip between 0 and umax=2. We will use this saturated value in the objective function to compute the cost of control.

In [3]:
def u_sat(u):
    return # ADD CODE HERE

def shoot_F(x0, U):
    x = x0.copy()
    X = [x.copy()]
    U_sat = []
    for u in U:
        # ADD CODE HERE
        # USE THE .append function for the lists X, U_sat to append new state and saturated control 
    return np.array(X), np.array(U_sat)

IndentationError: expected an indented block after 'for' statement on line 8 (932572751.py, line 11)

test your code

(10, 6) (9, 2) (Array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  2.54218831,
         0.        ],
       [ 0.        ,  0.25421883,  0.        ,  0.        ,  5.08437662,
         0.        ],
       [ 0.        ,  0.76265649,  0.        ,  0.        ,  7.62656494,
         0.        ],
       [ 0.        ,  1.52531299,  0.        ,  0.        , 10.16875325,
         0.        ],
       [ 0.        ,  2.54218831,  0.        ,  0.        , 12.71094156,
         0.        ],
       [ 0.        ,  3.81328247,  0.        ,  0.        , 15.25312987,
         0.        ],
       [ 0.        ,  5.33859546,  0.        ,  0.        , 17.79531818,
         0.        ],
       [ 0.        ,  7.11812727,  0.        ,  0.        , 20.3375065 ,
         0.        ],
       [ 0.        ,  9.15187792,  0.        ,  0.        , 22.87969481,
         0.        ]], dtype=float64), Array([[1.76159416, 1.76159416],
       [1.76159416, 1.76159416],
       [1.76159416, 1.76159416],
       [1.76159416, 1.76159416],
       [1.76159416, 1.76159416],
       [1.76159416, 1.76159416],
       [1.76159416, 1.76159416],
       [1.76159416, 1.76159416],
       [1.76159416, 1.76159416]], dtype=float64))


In [44]:
x0 = np.zeros(6)
U = np.ones((9,2))
print(
    shoot_F(x0,U)[0].shape, shoot_F(x0,U)[1].shape,
    shoot_F(x0,U)
)

(10, 6) (9, 2) (Array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  2.54218831,
         0.        ],
       [ 0.        ,  0.25421883,  0.        ,  0.        ,  5.08437662,
         0.        ],
       [ 0.        ,  0.76265649,  0.        ,  0.        ,  7.62656494,
         0.        ],
       [ 0.        ,  1.52531299,  0.        ,  0.        , 10.16875325,
         0.        ],
       [ 0.        ,  2.54218831,  0.        ,  0.        , 12.71094156,
         0.        ],
       [ 0.        ,  3.81328247,  0.        ,  0.        , 15.25312987,
         0.        ],
       [ 0.        ,  5.33859546,  0.        ,  0.        , 17.79531818,
         0.        ],
       [ 0.        ,  7.11812727,  0.        ,  0.        , 20.3375065 ,
         0.        ],
       [ 0.        ,  9.15187792,  0.        ,  0.        , 22.87969481,
         0.        ]], dtype=float64), Array([[1.7615941

## Q7 20 pts: Shooting-Based Objective Function

Here, we will need to redefine the cost function and terminal cost defined previously as MPC plans on smaller time-scales with a receding horizon (terminal time is then always moving away from the controller). As a result, we need to inform the controller on what the task is throughout planning (especially since MPC methods are simpler and need a lot more help to inform of solutions). We will define a quadratic cost with a terminal condition 
$$
    J = \frac{1}{N} \sum (x_k - x_d)^\top Q (x_k - x_d) + u_k^\top R u_k + (x_N - x_d)^\top Q_f (x_N - x_d)
$$
where $Q, Q_f, R$ are define below, and $N$ is the discrete time horizon (used to normalize the objective value).

In addition, this function will take in a control sequence an initial state and implicitly simulate the state trajectory using the shooting function and return a scalar loss value for the state/control trajectories.

In [45]:
_xd = np.array([0.,0.,2*np.pi,0.,0.,0.])
_Q = np.diag(np.array([2., 2., 80., .01, .01, .001]))
_Qf = np.diag(np.array([4., 4., 80., .01, .01, .01]))
_R = np.diag(np.array([0.0001, 0.0001]))
def soft_objective(x0, U):
    X, U_sat = shoot_F(x0, U)
    J = 0.0
    for x,u in zip(X[:-1], U_sat):
        # ADD CODE HERE
    return 

check your solution

3214.2475970870037 
 [[-1.06703508e+01  1.97283558e+01]
 [-7.96106806e+00  1.56823704e+01]
 [-5.67185427e+00  1.20607246e+01]
 [-3.79416819e+00  8.87195955e+00]
 [-2.31519799e+00  6.12888718e+00]
 [-1.21786119e+00  3.84858990e+00]
 [-4.80804735e-01  2.05242081e+00]
 [-7.84049178e-02  7.66003598e-01]
 [ 1.92325660e-02  1.92325660e-02]]


In [46]:
x0 = np.zeros(6)
U = np.ones((9,2))
print(
    soft_objective(x0, U), '\n',
    grad(soft_objective, argnums=1)(x0,U)
)

3214.2475970870037 
 [[-1.06703508e+01  1.97283558e+01]
 [-7.96106806e+00  1.56823704e+01]
 [-5.67185427e+00  1.20607246e+01]
 [-3.79416819e+00  8.87195955e+00]
 [-2.31519799e+00  6.12888718e+00]
 [-1.21786119e+00  3.84858990e+00]
 [-4.80804735e-01  2.05242081e+00]
 [-7.84049178e-02  7.66003598e-01]
 [ 1.92325660e-02  1.92325660e-02]]


## Q8 20 pts: MPC formulation 

Here, we will form the MPC controller. Recall that the control input needs to be time shifted. This will be done using an inplace assignment based on the jax syntax `U = U.at[:-1].set(U[1:])`, fill in the algorithm to update the control using the step_size variable as a first-order method as done above. 

In [47]:
jitted_grad_soft_obj = jit(grad(soft_objective, argnums=1))
# evaluate this one to compile
jitted_grad_soft_obj(x0,U)

def mpc(x0, U, step_size=1e-3):
    # shift the controls
    # ADD CODE HERE
    for i in range(4_000):
        # ADD CODE HERE

        # check if |dU| is small
        if np.linalg.norm(dU) < 1e-3:
            break
    return U

check your solution
[[ 3.42343083 -3.5459795 ]
 [ 3.28741085 -3.42092557]
 [ 3.13115721 -3.27728003]
 [ 2.94804728 -3.10820828]
 [ 2.7276021  -2.90206923]
 [ 2.45177566 -2.63638718]
 [ 2.08636243 -2.25699453]
 [ 1.56687352 -1.5455861 ]
 [ 0.96022509  0.96125492]]


In [48]:
x0 = np.zeros(6)
U = np.ones((9,2))
print(
    mpc(x0, U)
)

[[ 3.42343083 -3.5459795 ]
 [ 3.28741085 -3.42092557]
 [ 3.13115721 -3.27728003]
 [ 2.94804728 -3.10820828]
 [ 2.7276021  -2.90206923]
 [ 2.45177566 -2.63638718]
 [ 2.08636243 -2.25699453]
 [ 1.56687352 -1.5455861 ]
 [ 0.96022509  0.96125492]]


## Q9 10 pts: Animate and run the algorithm in real-time. 

Run the MPC in the loop using the dynamics F(x, u) as a stepping function with MPC generating controls. Recall that you only apply the first control U[0] to the dynamics. Render what the drone is doing as you step forward the sim. 

In [49]:
viz = meshcat.Visualizer()

drone  = viz["drone"]
drone_body = drone["body"]
drone_body.set_object(
    geom.Box([0.1,0.5,0.02])
)
drone_propFL = drone["propFL"]
drone_propFL.set_transform(tfm.translation_matrix([0.,-0.25,0.05])@tfm.rotation_matrix(np.pi/2,[1,0,0]))
drone_propFL.set_object(
    geom.Cylinder(height=0.01, radius=0.2)
)

drone_propFR = drone["propFR"]
drone_propFR.set_transform(tfm.translation_matrix([0.,0.25,0.05])@tfm.rotation_matrix(np.pi/2,[1,0,0]))
drone_propFR.set_object(
    geom.Cylinder(height=0.01, radius=0.2)
)
viz.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7005/static/


Write Code Here

In [50]:
tH = 2
N = int(tH/_dt)
x0 = np.zeros(6)
U = np.zeros((N-1, 2)) 

for t in range(100):
    # ADD CODE HERE 
    q, qdot = np.split(x0, 2)
    drone.set_transform(
        tfm.translation_matrix([0,q[0],q[1]]) @ tfm.rotation_matrix(q[2],[1,0,0])
    )
    time.sleep(_dt)


## Extra Credit on any assignment: MPC for Cartpole swingup + balance
write an mpc stabilizing controller for the cart pendulum pendulum is in the upright position following the open-loop control `U[t]`. Visualize the results below.